## Pororo
RTT 데이터들은 train_dataset 중 train 데이터(총 3952개)의 question만 RTT하여 제작하였음

In [1]:
# 원래 데이터셋 불러오기
from datasets import load_from_disk
raw_dataset = load_from_disk("../data/train_dataset/")
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 240
    })
})

In [2]:
raw_dataset = raw_dataset['train']
raw_dataset

Dataset({
    features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
    num_rows: 3952
})

### Papago RTT 데이터 불러오기

In [3]:
import pandas as pd
prr = pd.read_csv('trainset_rtt_pororo.csv')
prr.head()

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,어떤 국가기관이 대통령을 포함한 미국 행정부를 견제할 권리가 있는가?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대 인사 운영의 출발점이 어떤 책이 됐을까?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 누구를 위해 1717년에 썼나요?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",보통 11세기와 12세기에 전국의 특성이 어떤 것이냐는 것이다.,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146,992
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,그 이름으로 작성된 유물의 전체 병은 몇 개입니까?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334,548


### 데이터 라벨링 (잘된 것/이상한 것)

In [4]:
data_len = len(prr)

good = []
bad = []

for i in range(data_len):
    id = prr.iloc[i]['id']
    if id != raw_dataset[i]['id']:
        print(f"[iter {i}] prr id: {id}, raw_dataset id: {raw_dataset[i]['id']}")
        breakpoint()
    print('context: ')
    print('\t', prr.iloc[i]['context'])
    print('raw_dataset question: ')
    print('\t', raw_dataset[i]['question'])
    
    print('prr question: ')
    print('\t', prr.iloc[i]['question'])
    
    choice = input('Great or Bad?(Enter g or b):')
    if choice.lower() == 'g':
        good.append(i)
    elif choice.lower() == 'b':
        bad.append(i)
    else: pass

    if len(good) >= 10 and len(bad) >= 10:
        print('You got good/bad RTT data over 10!')
        break
    print('-'*50)

context: 
	 미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\n\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\n\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05
raw_dataset question: 
	 대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?
prr question: 
	 어떤 국가기관이 대통령을 포함한 미국 행정부를 견제할 권리가 있는가?
-----------

KeyboardInterrupt: Interrupted by user

In [5]:
print(good)
print(bad)

# 혹시나 날아갈까봐 복붙...
# good = [0, 19, 28, 32, 33, 42, 43, 49, 56, 73]
# bad = [3, 4, 5, 6, 7, 9, 11, 12, 13, 15, 16, 18, 20, 21, 22, 24, 25, 27, 29, 30, 31, 34, 36, 38, 40, 41, 44, 45, 46, 47, 48, 51, 52, 53, 54, 55, 57, 58, 59, 61, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 74, 75, 76]

[0, 19, 28, 32, 33, 42, 43, 49, 56, 73]
[3, 4, 5, 6, 7, 9, 11, 12, 13, 15, 16, 18, 20, 21, 22, 24, 25, 27, 29, 30, 31, 34, 36, 38, 40, 41, 44, 45, 46, 47, 48, 51, 52, 53, 54, 55, 57, 58, 59, 61, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 74, 75, 76]


In [6]:
good_prr = pd.DataFrame(columns=prr.columns) # 빈 데이터 프레임 생성
bad_prr = pd.DataFrame(columns=prr.columns)

good = good[:10] # 10개만 시도
bad = bad[:10]

for k in good:
    good_prr = good_prr.append(prr.iloc[k])
    
for j in bad:
    bad_prr = bad_prr.append(prr.iloc[j])

In [7]:
good_prr = good_prr.reset_index(drop=True)
good_prr.tail(2)

,title,context,question,id,answers,document_id,__index_level_0__
8,명탐정 코난: 천국으로의 카운트다운,니시타마 시로 캠프를 갔다 돌아오던 소년탐정단. 돌아오는 길에 새로 새워진 대형 건...,트윈타워 살인 현장에서 어떤 증거가 있었나요?,mrc-1-001136,"{'answer_start': [342], 'text': ['깨진 술잔']}",16753,747
9,이스라엘 독립 선언서,"독립 선언식은 텔 아비브 박물관 (오늘날엔 독립기념관)에서 열릴 예정이었으나, 영국...",경찰관이 티켓을 떼지 않은 이유는 무엇입니까?,mrc-0-005064,"{'answer_start': [481], 'text': ['독립 선언식']}",21063,3295


In [8]:
bad_prr = bad_prr.reset_index(drop=True)
bad_prr.tail(2)

,title,context,question,id,answers,document_id,__index_level_0__
8,장면,장면은 귀국전 국무총리에 임명되었으나 이듬해인 1951년 2월에 귀국한 후 정식으로...,그 장면이 한국으로 돌아온 지 몇 년이 되었나요?,mrc-0-001373,"{'answer_start': [26], 'text': ['1951년']}",6930,899
9,권번,권번은 주식회사 제도로 운영되었으며 일제가 그 관리를 주도하였다. 일제가 권번을 관...,권분 식당에 상장된 기생충의 이름은 무엇입니까?,mrc-1-000732,"{'answer_start': [524], 'text': [""'초일기'""]}",47810,471


### kobert로 임베딩을 만들어보자

In [66]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-1mo2ajf9/kobert-tokenizer
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4627 sha256=e96cdc5e4c44a0dde8d9cab60a2d47720f752adeeb686718f59cb09df82f6cf7
  Stored in directory: /tmp/pip-ephem-wheel-cache-hjs1ifbf/wheels/f7/cb/29/1a737fe71e5108dc30b04ea4a990f78ed271fa537aaf3fce7c
Successfully built kobert-tokenizer


In [9]:
import torch
from transformers import BertModel
from kobert_tokenizer import KoBERTTokenizer

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertModel.from_pretrained('skt/kobert-base-v1')

In [10]:
# question을 string으로 구성된 리스트로 넣기
good_prr_q = good_prr['question'].to_list()

In [11]:
tokenized_good_prr_q = tokenizer.batch_encode_plus(good_prr_q, padding=True)

In [12]:
good_prr_out = model(input_ids = torch.tensor(tokenized_good_prr_q['input_ids']),
            attention_mask = torch.tensor(tokenized_good_prr_q['attention_mask']))

In [13]:
good_prr_vec = good_prr_out.pooler_output

In [14]:
# raw data (reference) 가져오기
good_ref = []
bad_ref = []

for i in good:
    good_ref.append(raw_dataset[i]['question'])

for k in bad:
    bad_ref.append(raw_dataset[k]['question'])

In [15]:
tokenized_good_ref = tokenizer.batch_encode_plus(good_ref, padding=True)
good_ref_out = model(input_ids = torch.tensor(tokenized_good_ref['input_ids']),
            attention_mask = torch.tensor(tokenized_good_ref['attention_mask']))

In [16]:
good_ref_vec = good_ref_out.pooler_output

In [17]:
# from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

In [18]:
for i in range(10):
    print(f'{i}th sentence cosine similarity: ', cos_sim(good_prr_vec[i].detach().numpy(), good_ref_vec[i].detach().numpy()))

0th sentence cosine similarity:  0.9306401
1th sentence cosine similarity:  0.9894254
2th sentence cosine similarity:  0.98738754
3th sentence cosine similarity:  0.94703275
4th sentence cosine similarity:  0.9788575
5th sentence cosine similarity:  0.91562384
6th sentence cosine similarity:  0.38795048
7th sentence cosine similarity:  0.87369955
8th sentence cosine similarity:  0.9397687
9th sentence cosine similarity:  0.8875707


In [19]:
check_index = 6
print('reference sentence:  ', good_ref[check_index])
print('papago rtt sentence: ', good_prr_q[check_index])

reference sentence:   사후에 영장을 청구할 수 있는 예외조항이 존재하는 법 조항은?
papago rtt sentence:  사후 청구가 가능한 예외가 있는 법 조항은 무엇인가?


### bad도 해보자

In [20]:
bad_prr_q = bad_prr['question'].to_list()
tokenized_bad_prr_q = tokenizer.batch_encode_plus(bad_prr_q, padding=True)
bad_prr_out = model(input_ids = torch.tensor(tokenized_bad_prr_q['input_ids']),
            attention_mask = torch.tensor(tokenized_bad_prr_q['attention_mask']))
bad_prr_vec = bad_prr_out.pooler_output

In [21]:
tokenized_bad_ref = tokenizer.batch_encode_plus(bad_ref, padding=True)
bad_ref_out = model(input_ids = torch.tensor(tokenized_bad_ref['input_ids']),
            attention_mask = torch.tensor(tokenized_bad_ref['attention_mask']))
bad_ref_vec = bad_ref_out.pooler_output

In [22]:
for i in range(10):
    print(f'{i}th sentence cosine similarity: ', cos_sim(bad_prr_vec[i].detach().numpy(), bad_ref_vec[i].detach().numpy()))

0th sentence cosine similarity:  -0.071237616
1th sentence cosine similarity:  0.9647542
2th sentence cosine similarity:  0.74132144
3th sentence cosine similarity:  0.8985207
4th sentence cosine similarity:  0.95630556
5th sentence cosine similarity:  0.8534669
6th sentence cosine similarity:  0.9190687
7th sentence cosine similarity:  0.5533344
8th sentence cosine similarity:  0.7239824
9th sentence cosine similarity:  0.8218935


In [24]:
check_index = 1
print('reference sentence:  ', bad_ref[check_index])
print('papago rtt sentence: ', bad_prr_q[check_index])

reference sentence:   명문이 적힌 유물을 구성하는 그릇의 총 개수는?
papago rtt sentence:  그 이름으로 작성된 유물의 전체 병은 몇 개입니까?


In [25]:
bad_prr_q

['보통 11세기와 12세기에 전국의 특성이 어떤 것이냐는 것이다.',
 '그 이름으로 작성된 유물의 전체 병은 몇 개입니까?',
 '카드모스 남자들이 어디로 갔지?',
 '관을 태우려던 사람은 누구입니까?',
 '재난 속에서 무기와 장비를 버린 사람들이 한 무리인가요?',
 '고려왕은 어떤 마을에서 아내의 가족과 아내를 피난처로 만들었을까?',
 '벨레미아는 빅터의 일을 어떻게 생각해?',
 '업핑가스 왕의 이름이 어디서 나왔나요?',
 '그 장면이 한국으로 돌아온 지 몇 년이 되었나요?',
 '권분 식당에 상장된 기생충의 이름은 무엇입니까?']